In [0]:
from pyspark.sql.functions import explode, col, current_timestamp, coalesce, lower, regexp_extract


In [0]:
dbutils.fs.ls("abfss://landing@acmemcd.dfs.core.windows.net")

In [0]:
marketing_df = spark.read.format("csv").option("header", "true").load("abfss://landing@acmemcd.dfs.core.windows.net/marketing")

In [0]:
marketing_df.printSchema()
display(marketing_df)

In [0]:
silver_df = (marketing_df
    .select(
    col("lead_id").cast("string").alias("lead_id"),
    lower(col("email").cast("string")).alias("email"),
    lower(col("first_name").cast("string")).alias("first_name"),
    lower(col("last_name").cast("string")).alias("last_name"),
    col("country_code").cast("string").alias("country_code"),
    lower(col("region").cast("string")).alias("region"),
    col("utm_campaign").cast("string").alias("utm_campaign"),
    col("created_at").cast("timestamp").alias("created_at"),
    col("dt").cast("date").alias("dt"),
    current_timestamp().alias("ingestion_date")
    )
)
display(silver_df)

In [0]:
invalid_condition = (
    col("lead_id").isNull() |
    col("email").isNull() |
    col("country_code").isin("XX")
)

In [0]:
valid_df = silver_df.filter(~invalid_condition)
display(valid_df)

In [0]:
invalid_df = silver_df.filter(invalid_condition)
display(invalid_df)

In [0]:
%sql
drop table if exists `acme-dev`.silver.customers_linkage_full;